This is a starter notebook for the [Kitchenware classification](https://www.kaggle.com/competitions/kitchenware-classification) competition on Kaggle

To get started:

- Join the competition and accept rules
- Download your Kaggle credentials file
- If you're running in Saturn Cloud, configure your instance to have access to access the kaggle credentials

When this is done, we can download the data. We need to execute the following cell only once

In [ ]:
!kaggle competitions download -c kitchenware-classification
!mkdir data
!unzip kitchenware-classification.zip -d data > /dev/null
!rm kitchenware-classification.zip

In [ ]:
!ls

In [ ]:
#!unzip data/images.zip -d data

Now let's train a baseline model

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
tf.__version__

First, we will load the training dataframe and split it into train and validation

In [ ]:
df_train_full = pd.read_csv('data/train.csv', dtype={'Id': str})
df_train_full['filename'] = 'data/images/' + df_train_full['Id'] + '.jpg'
df_train_full.head()

In [ ]:
df_train_full.info()

In [ ]:
#splitting train_full in train(0.8) and val(0.2)
val_cutoff = int(len(df_train_full) * 0.8)
df_train = df_train_full[:val_cutoff]
df_val = df_train_full[val_cutoff:]

Now let's create image generators

In [ ]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_dataframe(
    df_train,
    x_col='filename',
    y_col='label',
    target_size=(150, 150),
    batch_size=32,
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_generator = val_datagen.flow_from_dataframe(
    df_val,
    x_col='filename',
    y_col='label',
    target_size=(150, 150),
    batch_size=32,
)

In [ ]:
def make_model(learning_rate):
    base_model = Xception(
        weights='imagenet',
        input_shape=(150, 150, 3),
        include_top=False
    )

    base_model.trainable = False

    inputs = keras.Input(shape=(150, 150, 3))

    base = base_model(inputs, training=False)
    vector = keras.layers.GlobalAveragePooling2D()(base)
    outputs = keras.layers.Dense(6)(vector)

    model = keras.Model(inputs, outputs)
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )
    
    return model

#### starter parameters were lr = 0.01 and epochs = 2 ####

In [ ]:
model = make_model(learning_rate = 0.001)
history_0_001 = model.fit(train_generator, epochs=10, validation_data=val_generator)

In [ ]:
plt.figure(figsize=(6, 4))

epochs = history_0_001.epoch
val = history_0_001.history['val_accuracy']
train = history_0_001.history['accuracy']

plt.plot(epochs, val, color='black', linestyle='solid', label='validation')
plt.plot(epochs, train, color='black', linestyle='dashed', label='train')

plt.title('Xception v1, lr=0.001')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

plt.xticks(epochs)

plt.legend()


plt.savefig('xception_v1_0_001.svg')

plt.show()

In [ ]:
#Let's add one more layer - and a dropout between them

def make_model(learning_rate, droprate):
    base_model = Xception(
        weights='imagenet',
        input_shape=(150, 150, 3),
        include_top=False
    )

    base_model.trainable = False

    inputs = keras.Input(shape=(150, 150, 3))
    
    base = base_model(inputs, training=False)
    vector = keras.layers.GlobalAveragePooling2D()(base)

    inner = keras.layers.Dense(100, activation='relu')(vector)
    drop = keras.layers.Dropout(droprate)(inner)

    outputs = keras.layers.Dense(6)(drop)

    model = keras.Model(inputs, outputs)
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )
    
    return model

In [ ]:
model = make_model(learning_rate=0.001, droprate=0.5)

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "xception_v2_0_5_{epoch:02d}_{val_accuracy:.3f}.h5",
        monitor="val_accuracy",
        save_best_only=True,
        mode='max'
    )
]

history_2 = model.fit(train_generator, epochs=15, validation_data=val_generator, callbacks=callbacks)

In [ ]:
epochs = history_2.epoch

train05 = history_2.history['accuracy']

val05 = history_2.history['val_accuracy']

In [ ]:
plt.figure(figsize=(6, 4))


plt.plot(epochs, val05, color='black', linestyle='solid', label='0.5')



plt.title('Xception')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')


plt.legend()

#plt.savefig('xception_v2_dropout.svg')

plt.show()

In [ ]:
plt.figure(figsize=(6, 4))


plt.plot(epochs, train05, color='black', linestyle='solid', label='0.5')



plt.title('Xception(train)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')


plt.legend()

#plt.savefig('xception_v2_dropout_train.svg')

plt.show()

### DATA AUGMENTATION ###

In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    zoom_range=0.1,
    horizontal_flip=True,
    #vertical_flip=True,
    rotation_range=5.0,
    fill_mode='nearest',
    #width_shift_range=0.1,
    #height_shift_range=0.1
    #channel_shift_range=0.2
    shear_range=0.2
)

train_generator = train_datagen.flow_from_dataframe(
    df_train,
    x_col='filename',
    y_col='label',
    target_size=(150, 150),
    batch_size=32,
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_generator = val_datagen.flow_from_dataframe(
    df_val,
    x_col='filename',
    y_col='label',
    target_size=(150, 150),
    batch_size=32,
)

In [ ]:
model = make_model(learning_rate=0.001, droprate=0.5)

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "xception_v3_0_5_{epoch:02d}_{val_accuracy:.3f}.h5",
        monitor="val_accuracy",
        save_best_only=True,
        mode='max'
    )
]

history_3 = model.fit(train_generator, epochs=10, validation_data=val_generator, callbacks=callbacks)

In [ ]:
epochs = history_3.epoch

train05 = history_3.history['accuracy']

val05 = history_3.history['val_accuracy']

In [ ]:
plt.figure(figsize=(6, 4))


plt.plot(epochs, val05, color='black', linestyle='solid', label='0.5')



plt.title('Xception')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')


plt.legend()

#plt.savefig('xception_v2_dropout.svg')

plt.show()

In [ ]:
plt.figure(figsize=(6, 4))


plt.plot(epochs, train05, color='black', linestyle='solid', label='0.5')



plt.title('Xception(train)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')


plt.legend()

#plt.savefig('xception_v2_dropout_train.svg')

plt.show()

### FINE TUNING ###

In [ ]:
base_model = Xception(
    weights='imagenet',
    input_shape=(150, 150, 3),
    include_top=False
)

base_model.trainable = False

inputs = keras.Input(shape=(150, 150, 3))
    
base = base_model(inputs, training=False)
vector = keras.layers.GlobalAveragePooling2D()(base)

inner = keras.layers.Dense(100, activation='relu')(vector)
drop = keras.layers.Dropout(rate=0.5)(inner)

outputs = keras.layers.Dense(6)(drop)

model = keras.Model(inputs, outputs)
    
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)
    

In [ ]:
history_4 = model.fit(train_generator, epochs=10, validation_data=val_generator)

In [ ]:
#unfreeze the last 32 layers of the base model for doing a partial fine tuning 
base_model.trainable = True
for layer in base_model.layers[:-32]:
  layer.trainable = False

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),  # Low learning rate to avoid desstruction of the learned weights
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "xception_capstone_{epoch:02d}_{val_accuracy:.3f}.h5",
        monitor="val_accuracy",
        save_best_only=True,
        mode='max'
    )
]

history_5 = model.fit(train_generator, epochs=10, validation_data=val_generator, callbacks=callbacks)

In [ ]:
epochs = history_5.epoch

train = history_5.history['accuracy']

val = history_5.history['val_accuracy']

In [ ]:
plt.figure(figsize=(6, 4))


plt.plot(epochs, val, color='black', linestyle='solid')



plt.title('Xception')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')


plt.legend()

#plt.savefig('xception_capstone_val_acc.svg')

plt.show()

In [ ]:
plt.figure(figsize=(6, 4))


plt.plot(epochs, train, color='black', linestyle='solid')



plt.title('Xception(train)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')


plt.legend()

#plt.savefig('xception_capstone_train_acc.svg')

plt.show()

### Training a larger model ###

In [ ]:
input_size = 299

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_dataframe(
    df_train,
    x_col='filename',
    y_col='label',
    target_size=(input_size, input_size),
    batch_size=32,
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_generator = val_datagen.flow_from_dataframe(
    df_val,
    x_col='filename',
    y_col='label',
    target_size=(input_size, input_size),
    batch_size=32,
)

In [ ]:
base_model = Xception(
    weights='imagenet',
    input_shape=(input_size, input_size, 3),
    include_top=False
)

base_model.trainable = False

inputs = keras.Input(shape=(input_size, input_size, 3))
    
base = base_model(inputs, training=False)
vector = keras.layers.GlobalAveragePooling2D()(base)

inner = keras.layers.Dense(100, activation='relu')(vector)
drop = keras.layers.Dropout(rate=0.5)(inner)

outputs = keras.layers.Dense(6)(drop)

model = keras.Model(inputs, outputs)
    
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [ ]:
model.fit(train_generator, epochs=5, verbose=1, validation_data=val_generator)

In [ ]:
#unfreeze the last 32 layers of the base model for doing a partial fine tuning 
base_model.trainable = True
for layer in base_model.layers[:-32]:
  layer.trainable = False

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-5),  # Low learning rate to avoid destruction of the learned weights
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "xception_final_{epoch:02d}_{val_accuracy:.3f}.h5",
        monitor="val_accuracy",
        verbose=1,
        save_best_only=True,
        mode='max'
    ),
    keras.callbacks.EarlyStopping(
        monitor='val_loss', 
        mode='min', 
        verbose=1, 
        patience=5
    )
]

history_6 = model.fit(train_generator, epochs=50, verbose=0, validation_data=val_generator, callbacks=callbacks)

In [ ]:
epochs = history_6.epoch

train = history_6.history['accuracy']

val = history_6.history['val_accuracy']

In [ ]:
plt.figure(figsize=(6, 4))


plt.plot(epochs, val, color='black', linestyle='solid')



plt.title('Xception')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')


plt.legend()

#plt.savefig('xception_capstone_val_acc.svg')

plt.show()

In [ ]:
plt.figure(figsize=(6, 4))


plt.plot(epochs, train, color='black', linestyle='solid')



plt.title('Xception(train)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')


plt.legend()

#plt.savefig('xception_capstone_train_acc.svg')

plt.show()

Now let's use this model to predict the labels for test data

In [ ]:
#model = keras.models.load_model('xception_v2_0_5_14_0.906.h5') #best result case with 1 more layer and dropout

#model = keras.models.load_model('xception_v3_0_5_15_0.902.h5') #1 more layer, dropout and augmentation
#model = keras.models.load_model('xception_v3_0_5_09_0.909.h5') #1 more layer, dropout and augmentation final

#model = keras.models.load_model('xception_v4_0_5_10_0.922.h5') #1 more layer, dropout, augmentation and partial fine tuning

#model = keras.models.load_model('xception_capstone_08_0.925.h5') #final model for capstone (dropout, augmentation and partial tuning)

model = keras.models.load_model('xception_final_03_0.964.h5') #final model for kaggle (dropout, augmentation and partial tuning, increased input_size to 299)

In [ ]:
df_test = pd.read_csv('data/test.csv', dtype={'Id': str})
df_test['filename'] = 'data/images/' + df_test['Id'] + '.jpg'
df_test.head()

In [ ]:
input_size = 299

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

#class_mode = 'input'means that in the label arrays that are returned each label will be will be images identical to input images
#useful for fitting autoencoders

test_generator = test_datagen.flow_from_dataframe(
    df_test,
    x_col='filename',
    class_mode='input',
    #target_size=(150, 150),
    target_size=(input_size, input_size),
    batch_size=32,
    shuffle=False
)

In [ ]:
test_generator.filenames[:5]

In [ ]:
y_pred = model.predict(test_generator)

In [ ]:
y_pred.shape

In [ ]:
#dtype = '<U5' means unicode 5 characters with byte-order little-endian (<)
classes = np.array(list(train_generator.class_indices.keys()))
classes

In [ ]:
predictions = classes[y_pred.argmax(axis=1)]

In [ ]:
predictions[:5]

### BENTO ML ###

In [ ]:
import bentoml

In [ ]:
bentoml.keras.save_model("keras_xception_final", model)

## Finally, we need to prepare the submission ##

### submitted till now: 
1) lr=0.01 epochs=2 val_acc=0.8085 (scored on kaggle as 0.81860) \
2) lr=0.01 epochs=5 val_acc=0.8741 (scored as 0.88062) \
3) lr=0.01 epochs=10 val_accuracy=0.8822  (scored as? NOT submitted, overfitting  ) \
4) lr=0.1 epochs=10 val_accuracy=0.8831   (scored as ? NOT submitted, overfitting ) \
5) lr=0.001 epochs=10 val_accuracy=0.8867   (scored as ? NOT submitted, overfitting ) \
6) lr=0.001 epochs=15 val_accuracy=0.8948  (scored as ? NOT submitted, overfitting ) \
7) lr=0.001 epochs=15 val_accuracy=0.9006  (scored as 0.89302 ) \
8) lr=0.001 epochs=15, data_augmentation, val_accuracy=0.9020  (scored as 0.89819 ) \
9) lr=0.001 epochs=15, data_augmentation, val_accuracy=0.9065  (scored as 0.89819 ) \
10) lr=0.001 epochs=10, data_augmentation, partial fine tuning val_accuracy=0.9218  (scored as 0.92144) \
11) lr=0.001 epochs=50, data_augmentation, partial fine tuning, larger model (from 150 to 299) val_accuracy=0.96403  (scored as 0.96227) \

In [ ]:
df_submission = pd.DataFrame()
df_submission['filename'] = test_generator.filenames
df_submission['label'] = predictions

df_submission['Id'] = df_submission.filename.str[len('data/images/'):-4]
del df_submission['filename']

In [ ]:
df_submission.head()

In [ ]:
df_submission[['Id', 'label']].to_csv('submission.csv', index=False)

In [ ]:
val_acc = round(max(history_6.history['val_accuracy']), 4)
#val_acc = 0.9218
print(f'validation: {val_acc}')

In [ ]:
!kaggle competitions submit kitchenware-classification -f submission.csv -m f'validation: {val_acc}'